In [ ]:
!pip install openai

In [2]:
from openai import OpenAI
import json
import os
from datetime import datetime, timedelta
from google.colab import userdata

In [3]:
api_key = userdata.get('OPENAI_API_KEY')

### Agent Memory (Persistent State)

In [4]:
MEMORY_FILE = "memory.json"

def load_memory():
    if os.path.exists(MEMORY_FILE):
        with open(MEMORY_FILE, "r") as f:
            return json.load(f)
    return {}

def save_memory(memory):
    with open(MEMORY_FILE, "w") as f:
        json.dump(memory, f, indent=4)

### Core LLM Call Function

In [5]:

client = OpenAI(api_key=api_key)


In [6]:
response = client.chat.completions.create(
    model="gpt-5-nano",
    messages=[{"role": "user", "content": "Say OK if this works"}]
)

print(response.choices[0].message.content)


OK


In [7]:
def llm_call(system_prompt, user_prompt):
    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=1
    )
    return response.choices[0].message.content

### Goal Understanding Agent

In [8]:
def understand_goal(user_input):
    system_prompt = """
    You are an AI study planning agent.
    Extract the following fields and return ONLY valid JSON:
    {
      "exam_name": "",
      "total_days": number,
      "daily_hours": number,
      "subjects": []
    }
    """
    return llm_call(system_prompt, user_input)


### Syllabus Decomposition Agent

In [9]:
def decompose_syllabus(subjects):
    system_prompt = """
    You are an expert academic planner.
    Break each subject into 5–7 study topics.
    Return ONLY valid JSON.
    """
    return llm_call(system_prompt, json.dumps(subjects))


### Study Plan Generator Agent

In [10]:
def generate_weekly_study_plan(syllabus, total_days, daily_hours):
    total_weeks = max(1, total_days // 7)

    system_prompt = """
    You are an AI study planner.
    Create a WEEK-WISE study plan.

    Rules:
    - Each week should have:
        • subjects
        • topics
        • total_hours
        • revision_focus
    - Balance subjects across weeks
    - Include weekly revision
    - Return ONLY valid JSON in this format:

    {
      "Week 1": {
        "subjects": [],
        "topics": [],
        "total_hours": number,
        "revision_focus": ""
      }
    }
    """

    payload = {
        "syllabus": syllabus,
        "total_weeks": total_weeks,
        "daily_hours": daily_hours
    }

    return llm_call(system_prompt, json.dumps(payload))


### Progress Tracker Agent

In [11]:
def update_progress(day, completed=True):
    memory = load_memory()
    memory[str(day)] = {
        "completed": completed,
        "timestamp": str(datetime.now())
    }
    save_memory(memory)


### Reflection & Re-Planning Agent

In [12]:
def reflect_and_replan(plan, memory):
    system_prompt = """
    You are a reflection agent.
    If tasks were missed, adjust future days.
    Reduce overload and reschedule topics.
    Return ONLY updated JSON plan.
    """
    payload = {
        "plan": plan,
        "progress": memory
    }
    return llm_call(system_prompt, json.dumps(payload))


# Run the Agent

In [13]:
user_input = """
I am preparing for TNPSC Group 4 exam.
I have 45 days.
I can study 4 hours per day.
Subjects: Tamil, General Studies, Aptitude
"""

goal = understand_goal(user_input)
goal_data = json.loads(goal)

syllabus = decompose_syllabus(goal_data["subjects"])
syllabus_data = json.loads(syllabus)

study_plan = generate_weekly_study_plan(
    syllabus_data,
    goal_data["total_days"],
    goal_data["daily_hours"]
)

study_plan_data = json.loads(study_plan)
study_plan_data


{'Week 1': {'subjects': ['Tamil', 'General Studies', 'Aptitude'],
  'topics': ['Tamil Grammar and Usage',
   'Tamil Vocabulary and Usage',
   'History of India: Ancient to Modern',
   'Geography: Physical and Indian Geography',
   'Numerical Reasoning and Data Interpretation',
   'Percentages, Ratios and Proportions'],
  'total_hours': 28,
  'revision_focus': 'Weekly revision: consolidate Tamil basics, core General Studies concepts, and Aptitude fundamentals; include 20 mixed-question practice.'},
 'Week 2': {'subjects': ['Tamil', 'General Studies', 'Aptitude'],
  'topics': ['Tamil Literature: Classics and Modern Works',
   'Comprehension and Essay Writing in Tamil',
   'Constitution and Governance',
   'Economy and Social Development',
   'Time, Speed and Distance',
   'Simple and Compound Interest'],
  'total_hours': 28,
  'revision_focus': 'Revision focus: revisit Week 1 topics, reinforce comprehension and governance concepts, and solve 20-30 practice questions across all three subj

In [15]:
def update_week_progress(week, completed=True):
    memory = load_memory()
    memory[f"Week {week}"] = {
        "completed": completed,
        "timestamp": str(datetime.now())
    }
    save_memory(memory)


In [17]:
update_week_progress(week=1, completed=False)

In [18]:
print(load_memory())

{'Week 1': {'completed': False, 'timestamp': '2026-01-30 17:52:13.305231'}}


In [19]:
update_week_progress(week=2, completed=True)
print(json.dumps(load_memory(), indent=4))

{
    "Week 1": {
        "completed": false,
        "timestamp": "2026-01-30 17:52:13.305231"
    },
    "Week 2": {
        "completed": true,
        "timestamp": "2026-01-30 17:54:04.710776"
    }
}
